In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *


#Instructions for the pipeline
Requires two inputs for training:
    - Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration)
    - NetsurfP and Biopython data that has been precalculated
    - X characteristics to predict

pipeline
Take mass spec spreadsheet
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration
Merge with Proteome data to get file that has
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,Sequence
Calculate protein features using biopython
Merge with NSP data to get all protein features

Split into X and Y dataset with Entries as labels

In [2]:
### New Data workup for RFG

# Pull together Proteomic data
in_dir="Input_data/Proteomic data/abundance/"
#combine Mass Spec data input into one excel spreadsheet - Entry - Abundance labeled by NP Unique ID
#Abundance as a percent
#take files in_dir and combine then into one pandas df (raw_MS_data)
files = os.listdir(in_dir)
for i,f in enumerate(files):
    if i==0:
        raw_MS_data=pd.read_excel(in_dir+f,header=0)
    else:
        temp = pd.read_excel(in_dir+f,header=0)
        raw_MS_data=raw_MS_data.merge(temp,how='outer',on='Entry')
# melt the df to make it an accession number, NPUNID, Abundance dataset
raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'],var_name='NPUNID', value_name='Abundance')
#remove prots that were added due to merge
raw_MS_data=raw_MS_data.dropna()

In [8]:
###Bring in controls (MS data for serums)##
#Controls is the abundance of the proteins in serum that have been observed via proteomics
#look at other sources here for human proteins, but internal sources for BALF and FBS
controls=pd.read_excel('Input_data/Proteomic data/controls_combined.xlsx',header=0)
MS_data_controls = pd.merge(raw_MS_data,controls,how='inner', on='Entry')
###Bring in Uniprot_data,NSPdata and NP data##
uniprot_filepath='Input_data/BioPython_data/Combined_biopyCalcs.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
NSPfilePath='Input_data/NetSurfP_data/Combined.xlsx'
NSP_data=pd.read_excel(NSPfilePath)
###Bring in NP data and merge to get complete NP dataset###
NP_filepath='Input_data/NPs/NP_Database.xlsx'
NPUNdata=pd.read_excel(NP_filepath,header=0,sheet_name='NPUNID')
NPprop=pd.read_excel(NP_filepath,header=0,sheet_name='NP_Props')
NPdata=pd.merge(NPUNdata,NPprop,how="left",on='NPID')

In [9]:
#calculate Enrichment
#####MAYBE add binning here to to keep negative results and improve capapbilities######
MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
raw_prop_data=pd.merge(MS_data, uniprot_dat, how='left',on='Entry')

C:\Users\kmp95\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
MS_data_clean = raw_MS_data.copy()
PROT_cleaned_data = normalize_mass_length_1DF(raw_prop_data) #function found in data_prep_functions line 167, normalizes mass, length and mw by dividing all values by the max in the column
Protein_data_complete = pd.merge(PROT_cleaned_data, NSP_data, left_on='Entry', right_on='Entry') #merges netsurfp features and biopython features
#creates new column called asa_sum_normalized which is the asa_sum value divide by the mass of the protein
for df in [Protein_data_complete]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

data_complete= pd.merge(Protein_data_complete,NPdata,how='left', on='NPUNID')
data_complete.drop(columns=['notes','Notes','NPUNID'],inplace=True)
#Optional here to drop all enrichment values that are NA - also have to deal with positive/negative infinity values
# data_complete.dropna(subset=['Enrichment'],inplace=True)

In [ ]:
# data_complete.drop(columns='Unnamed: 0',inplace=True)

In [14]:
data_complete

,Entry,Enrichment,Sequence,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,...,Dtem,Dh_core,Dh_functionalized,Shaken,Centrifuged,Protein Source,NP_incubation Concentration (mg/mL),Incubation Concentration (mg/ml),Incubation Time (minutes),Temperature
0,P02769,-2.532041,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,100.0,230.0,230.0,1.0,0.0,FBS,3.2,4.0,30.0,25.0
1,P02769,-2.100316,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,100.0,230.0,230.0,1.0,0.0,FBS,3.2,4.0,30.0,25.0
2,P02769,-2.134677,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,100.0,230.0,230.0,1.0,0.0,FBS,3.2,4.0,30.0,25.0
3,P02769,-0.343872,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,40.5,2083.0,2083.0,0.0,1.0,FBS,4.0,4.0,30.0,25.0
4,P02769,-2.16923,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,188.0,353.9,353.9,0.0,1.0,FBS,4.0,4.0,30.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,Q00896,-12,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,800.0,680.0,680.0,1.0,1.0,BALF,125.0,0.2,60.0,25.0
3078,Q00896,-12,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,378.0,410.0,410.0,1.0,1.0,BALF,62.5,0.2,60.0,25.0
3079,Q00896,1.789023,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,378.0,410.0,410.0,1.0,1.0,BALF,62.5,0.2,60.0,25.0
3080,Q00896,-12,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,378.0,410.0,410.0,1.0,1.0,BALF,62.5,0.2,60.0,25.0


In [13]:
data_complete.dropna(inplace=True)
#replace -infintiy and positive infinity with -12 and 12

#####COME BACK AND FIND BETTER SOLUTION!!!##
data_complete= data_complete.replace([-np.inf],'-12')
data_complete=data_complete.replace([np.inf],'12')

In [ ]:
#set labels (what we are trying to predict) as Enrichment column
labels=data_complete['Enrichment'].copy()
#make it one dimenisional
labels=np.ravel(labels)
#drop qualitative, not neccessary, and label columns
df=data_complete.drop(['Entry','Sequence','Core Material','Ligand','NPID','Enrichment'],axis=1)

# print(df)
#these are left over metrics from the helper functions from landry paper replace with better options for looking at metrics###
first_frame = True #starting dataframe for saving metrics
correctness_frame = pd.DataFrame()
metrics_frame = pd.DataFrame()
print_metrics = 1 #0, doesn't show metrics while runnning for each model, 1 does show metrics


In [ ]:
labels

In [ ]:
data_complete

In [ ]:
#Run recursive feature elimination to determine top features to select
#currently selecting 15 although that is arbitrary
### COME UP with better more objective way for determining number of features####
from sklearn.feature_selection import RFE

estimator = RandomForestRegressor(n_estimators=100)
selector = RFE(estimator, n_features_to_select= 15, step=1)
selector = selector.fit(df,labels)
selector.support_

In [ ]:
selector.ranking_

In [ ]:
feat_list=selector.get_feature_names_out()
print(feat_list)

In [ ]:
df_rfe=df[feat_list].copy()
df_rfe

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

In [ ]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i])

In [22]:

x_train, x_test, y_train, y_test = train_test_split(df_rfe,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
# print(rfg.feature_importances_)

# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

0.43730337548323595


In [28]:
feat_importances=rfg.feature_importances_
# print(feat_importances)
print('feature: importance score')
for i,col in enumerate(df_rfe.columns):
    print(col,feat_importances[i])

feature: importance score
frac_aa_I 0.06596630314032861
frac_aa_Q 0.05914304079572
frac_aa_W 0.06973760909851004
molecular_weight 0.055950814276992854
flexibility_var 0.060526394166567826
flexibility_min 0.06685478674919744
fraction_exposed_polar_exposed 0.06073350567070096
rsa_std 0.06858982375397228
fraction_total_exposed_M 0.07155243384020243
fraction_total_exposed_Y 0.062404576064288855
fraction_exposed_exposed_A 0.04876247693133964
fraction_exposed_exposed_H 0.05787822204456944
fraction_exposed_exposed_T 0.061567447024784984
Zeta Potential 0.09787628088636142
NP_incubation Concentration (mg/mL) 0.09245628555646303
